In [29]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import google_key

In [7]:
#import CSV from WeatherPy as DataFrame
city_data_file = "../WeatherPy/cities.csv"

#read into df
city_data_df = pd.read_csv(city_data_file)

#get rid of index column that was read in
city_data_df = city_data_df[["City", "Lat", "Lng", "Max Temp", "Cloudiness", "Humidity", "Wind Speed",
                            "Country", "Date"]]
city_data_df.head()

,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,hilo,19.7297,-155.0900,73.40,90,83,6.26,US,1611782150
1,albany,42.6001,-73.9662,30.00,81,62,13.00,US,1611782082
2,bredasdorp,-34.5322,20.0403,57.20,96,72,5.75,ZA,1611782144
3,cockburn town,21.4612,-71.1419,80.60,40,54,10.36,TC,1611782640
4,atuona,-9.8000,-139.0333,78.48,2,78,15.17,PF,1611782151


In [9]:
#double check the data types of all numbers
city_data_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Cloudiness      int64
Humidity        int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [15]:
##Humidity Heatmap
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

#store city lat/lng and humidity into variables to use for heat map
locations = city_data_df[["Lat", "Lng"]]

#convert humidity from int to float
humidity = city_data_df[["Humidity"]].astype('float')

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

TypeError: float() argument must be a string or a number, not 'DataFrame'

In [21]:
## Create New DF to fit weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

# Narrow down DF to find cities with ideal weather conditions
# My ideal weather conditions: Max temp > 65 but less than 85, wind speed < 5 mph, less than 20% cloudiness
hotel_df = city_data_df.loc[(city_data_df["Max Temp"] > 65) & (city_data_df["Max Temp"] < 85) 
                                    & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 20)]
hotel_df.head()

,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
46,coolum beach,-26.5333,153.1000,80.01,11,82,3.00,AU,1611782647
54,cururupu,-1.8283,-44.8683,80.78,10,76,4.14,BR,1611782648
75,odweyne,9.4092,45.0640,65.75,0,41,4.45,SO,1611782651
86,oussouye,12.4850,-16.5469,77.00,0,88,2.30,SN,1611782413
89,hervey bay,-25.2986,152.8535,79.00,7,79,4.00,AU,1611782584


In [ ]:
## Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [42]:
#create blank column in data frame called Hotel Name
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "key": google_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_address = requests.get(base_url, params=params)
    
    #print hotel address url in order to check that it was called correctly
    #print(hotel_address.url)

    # convert to json
    hotel_address = hotel_address.json()
    
    #can print json to check syntax/where the name of the hotels is located
    #print(json.dumps(hotel_address, indent=4, sort_keys=True))

    # Incorporate a try-except to skip any cities that are missing a data point.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\erinc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\erinc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Missing field/result... skipping.


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
